In [1]:
import numpy as np
import pandas as pd

In [4]:
df_clv=pd.read_csv('df_clv.csv')
df_clv.head()

,age,gender,security_no,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_session_duration,avg_transaction_value,...,feedback_No reason specified,feedback_Poor Customer Service,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website,Churn_probability
0,0.148148,0,XW0DQ7H,17-08-2017,0,xxxxxxxx,16:08:02,0.991220,0.514859,0.526717,...,0,0,0,0,1,0,0,0,0,0.003610
1,0.407407,0,5K0N3X1,28-08-2017,0,CID21329,12:38:13,0.990244,0.515803,0.121456,...,0,0,0,0,0,1,0,0,0,0.003547
2,0.629630,0,1F2TCL3,11-11-2016,1,CID12313,22:53:21,0.988293,0.550486,0.204074,...,0,0,0,1,0,0,0,0,0,0.964750
3,0.500000,1,VJGJ33N,29-10-2016,1,CID3793,15:57:50,0.985366,0.473971,0.246577,...,0,0,0,1,0,0,0,0,0,0.970951
4,0.388889,0,SVZXCWB,12-09-2017,0,xxxxxxxx,15:46:44,0.994146,0.483866,0.238950,...,0,0,0,1,0,0,0,0,0,0.959872


In [9]:
df_clv.dtypes

age                                               float64
gender                                              int64
security_no                                        object
joining_date                                       object
joined_through_referral                             int64
referral_id                                        object
last_visit_time                                    object
days_since_last_login                             float64
avg_session_duration                              float64
avg_transaction_value                             float64
avg_frequency_login_days                          float64
points_in_wallet                                  float64
used_special_discount                               int64
offer_application_preference                        int64
past_complaint                                      int64
churn                                               int64
logins_90d                                        float64
active_days_90

In [10]:
df_clv['joining_date']=pd.to_datetime(df_clv['joining_date'])
df_clv['last_visit_time']=pd.to_datetime(df_clv['last_visit_time'])

/var/folders/nc/hvlqvddd2xj11jl0l69nx3580000gn/T/ipykernel_34094/3519070926.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clv['joining_date']=pd.to_datetime(df_clv['joining_date'])
/var/folders/nc/hvlqvddd2xj11jl0l69nx3580000gn/T/ipykernel_34094/3519070926.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clv['last_visit_time']=pd.to_datetime(df_clv['last_visit_time'])


In [12]:
df_clv.dtypes

age                                                      float64
gender                                                     int64
security_no                                               object
joining_date                                      datetime64[ns]
joined_through_referral                                    int64
referral_id                                               object
last_visit_time                                   datetime64[ns]
days_since_last_login                                    float64
avg_session_duration                                     float64
avg_transaction_value                                    float64
avg_frequency_login_days                                 float64
points_in_wallet                                         float64
used_special_discount                                      int64
offer_application_preference                               int64
past_complaint                                             int64
churn                    

In [47]:
df_clv['CLV']=1/(df_clv['days_since_active']+0.01/(df_clv['avg_transaction_value']*df_clv['avg_frequency_login_days']*df_clv['avg_session_duration']))
df_clv['CLV2']=df_clv['days_since_active']/(df_clv['avg_transaction_value']*df_clv['avg_frequency_login_days']*df_clv['avg_session_duration']+0.01)

In [52]:
df_clv[['CLV','CLV2']]

,CLV,CLV2
0,10.351928,0.169892
1,2.777300,0.330439
2,2.751969,2.802606
3,2.185596,4.293620
4,5.131746,0.371108
...,...,...
36987,3.862843,0.253164
36988,1.366204,8.968413
36989,2.495195,2.823628
36990,0.473287,0.000000


In [50]:
df_clv[['CLV','CLV2']].describe()

,CLV,CLV2
count,36992.000000,36992.000000
mean,5.519858,1.106420
std,4.033890,2.144398
min,0.000000,0.000000
25%,2.614723,0.057445
50%,4.714134,0.416449
75%,7.427570,1.187294
max,37.295092,47.995575


In [68]:
x_clv=df_clv.drop(columns=['referral_id','security_no','CLV','CLV2','joining_date','last_visit_time'])
y_clv=df_clv['CLV']

In [69]:
from sklearn.model_selection import train_test_split
x_train_clv,x_test_clv,y_train_clv,y_test_clv=train_test_split(x_clv,y_clv,test_size=0.2,random_state=42)

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


models={
    "LinearRegression":LinearRegression(),
    "RandomForest":RandomForestRegressor(n_estimators=200, random_state=42),
    "XGB":XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42),
    "LGBM":LGBMRegressor(n_estimators=300, learning_rate=0.1, random_state=42,verbose=-1),
    "CatBoost":CatBoostRegressor(iterations=300, learning_rate=0.1, verbose=False, random_state=42)
}


In [71]:
from sklearn.metrics import root_mean_squared_error,mean_absolute_error,r2_score

In [77]:
results={}

for name,model in models.items():
    model.fit(x_train_clv,y_train_clv)
    y_pred_clv=model.predict(x_test_clv)

    rmse=root_mean_squared_error(y_test_clv,y_pred_clv)
    mae=mean_absolute_error(y_test_clv,y_pred_clv)
    r2=r2_score(y_test_clv,y_pred_clv)

    results[name]={"RMSE":rmse,"MAE":mae,"R2_Score":r2}

In [78]:
results_df=pd.DataFrame(results)
results_df

,LinearRegression,RandomForest,XGB,LGBM,CatBoost
RMSE,1.705985,0.338262,0.195511,0.206038,0.158614
MAE,1.083959,0.128587,0.088140,0.091706,0.084629
R2_Score,0.823069,0.993044,0.997676,0.997419,0.998471


In [83]:
from sklearn.model_selection import cross_val_score
cross_val={}
for name,model in models.items():
    cross_validation=cross_val_score(model,x_train_clv,y_train_clv,cv=5,scoring='neg_mean_squared_error')
    cross_val[name]={"Cross_val_score":cross_validation.mean()}

In [84]:
scores_df=pd.DataFrame(cross_val)
scores_df

,LinearRegression,RandomForest,XGB,LGBM,CatBoost
Cross_val_score,-2.88489,-0.103539,-0.033764,-0.038692,-0.022218
